In [21]:
# Import necessary modules
from tkinter import *
import datetime
import time
import winsound

# Define the Alarm class
class Alarm:
    def __init__(self, master):
        # Initialize the class with the master window
        self.master = master
        self.master.title("Alarm Clock")
        self.master.geometry("400x500")
        
        # Lists and dictionaries to store alarm details and snooze timer IDs
        self.alarms = []  
        self.snooze_ids = {}  

        # Create the user interface
        self.create_widgets()

    def create_widgets(self):
        # Create and pack UI elements using Tkinter
        Label(self.master, text="Set Alarm", font=("Arial", 20, "bold"), bg="#3498db", fg="white").pack(pady=10)
        
        # Variables to store input values
        self.hour_var = StringVar()
        self.min_var = StringVar()
        self.sec_var = StringVar()
        self.label_var = StringVar()
        self.repeat_var = IntVar()

        # Labels and entry widgets for setting the alarm time
        Label(self.master, text="Hour:").pack()
        Entry(self.master, textvariable=self.hour_var, width=5).pack()

        Label(self.master, text="Minute:").pack()
        Entry(self.master, textvariable=self.min_var, width=5).pack()

        Label(self.master, text="Second:").pack()
        Entry(self.master, textvariable=self.sec_var, width=5).pack()

        # Label and entry for setting a label for the alarm
        Label(self.master, text="Label:").pack()
        Entry(self.master, textvariable=self.label_var, width=20).pack()

        # Checkbox for setting alarm repetition
        Checkbutton(self.master, text="Repeat", variable=self.repeat_var).pack()

        # Button to set a new alarm
        Button(self.master, text="Set Alarm", command=self.set_alarm, font=("Arial", 14, "bold"), bg="#2ecc71", fg="white").pack(pady=10)

        # Listbox to display existing alarms
        self.alarm_listbox = Listbox(self.master, selectmode=SINGLE, width=40, height=5)
        self.alarm_listbox.pack()

        # Button to delete a selected alarm
        Button(self.master, text="Delete Alarm", command=self.delete_alarm, font=("Arial", 14, "bold"), bg="#e74c3c", fg="white").pack()

        # Button to snooze a selected alarm
        Button(self.master, text="Snooze", command=self.snooze_alarm, font=("Arial", 14, "bold"), bg="#f39c12", fg="white").pack(pady=10)
        
    def set_alarm(self):
        # Get user input to set a new alarm
        hour = self.hour_var.get()
        minute = self.min_var.get()
        second = self.sec_var.get()
        label = self.label_var.get()
        repeat = bool(self.repeat_var.get())

        # Format the alarm time
        alarm_time = f"{hour.zfill(2)}:{minute.zfill(2)}:{second.zfill(2)}"

        # Create a dictionary with alarm details and append to the alarms list
        alarm_details = {
            'time': alarm_time,
            'label': label,
            'repeat': repeat,
            'active': True
        }
        self.alarms.append(alarm_details)

        # Update the displayed alarm list
        self.update_alarm_listbox()

    def delete_alarm(self):
        # Delete the selected alarm from the alarms list
        selected_index = self.alarm_listbox.curselection()
        if selected_index:
            del self.alarms[selected_index[0]]
            self.update_alarm_listbox()

    def update_alarm_listbox(self):
        # Update the displayed list of alarms in the listbox
        self.alarm_listbox.delete(0, END)
        for alarm in self.alarms:
            label = alarm['label'] if alarm['label'] else 'No Label'
            repeat_status = "Repeat" if alarm['repeat'] else "One-time"
            self.alarm_listbox.insert(END, f"{alarm['time']} - {label} ({repeat_status})")

    def alarm(self, set_alarm_time, label, repeat):
        # Check if it's time to trigger the alarm
        while True:
            time.sleep(1)
            current_time = datetime.datetime.now()
            now = current_time.strftime("%H:%M:%S")

            if now == set_alarm_time:
                # Print a message and play a sound
                print(f"Time to Wake up - {label}")
                winsound.PlaySound("sound.wav", winsound.SND_ASYNC)
                if not repeat:
                    # Disable the alarm if it's not set to repeat
                    self.disable_alarm(label)
                break

    def snooze_alarm(self):
        # Snooze the selected alarm for 5 minutes
        selected_index = self.alarm_listbox.curselection()
        if selected_index:
            snooze_time = datetime.datetime.now() + datetime.timedelta(minutes=5)
            snooze_time_str = snooze_time.strftime("%H:%M:%S")

            # Update the alarm time and the displayed alarm list
            self.alarms[selected_index[0]]['time'] = snooze_time_str
            self.update_alarm_listbox()

            label = self.alarms[selected_index[0]]['label']
            print(f"Alarm snoozed for 5 minutes - {label}")

            # Cancel the previous snooze timer if it exists
            if label in self.snooze_ids:
                self.master.after_cancel(self.snooze_ids[label])

            # Set a new snooze timer
            self.snooze_ids[label] = self.master.after(300000, self.play_snooze_sound, label)

    def play_snooze_sound(self, label):
        # Play the snooze sound asynchronously
        winsound.PlaySound("snooze.wav", winsound.SND_ASYNC)

    def disable_alarm(self, label):
        # Disable the specified alarm
        for alarm in self.alarms:
            if alarm['label'] == label:
                alarm['active'] = False

    def start_alarms(self):
        # Start the alarm checking loop
        while True:
            for alarm in self.alarms:
                if alarm['active']:
                    self.alarm(alarm['time'], alarm['label'], alarm['repeat'])
            time.sleep(1)

if __name__ == "__main__":
    # Create the Tkinter root window
    root = Tk()
    alarm_clock = Alarm(root)

    # Start a separate thread to handle alarms
    import threading
    alarm_thread = threading.Thread(target=alarm_clock.start_alarms)
    alarm_thread.start()

    # Start the Tkinter main event loop
    root.mainloop()
